# GameDay2021 Live Draft Example

See the GameDay_Notebook for examples of running simulations.  This page will focus in-depth on using the code in a live-draft situation.  

You need four components, all which are provided, but can be edited/substituted:
1. Projections
2. Rankings
3. Injuries
4. Excel spreadsheet of picks in real-time

#### Projections 
Made available for you via [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips) are ZiPS, Steamer, and TheBat projections.  Adding your own shouldn't be too hard, see the directories and data for formatting.  
#### Rankings 
Available rankings were scraped from Yahoo and FantasyPros, but adding your own is simple and could provide an edge; case in point, I'm working on improved, simulation-based rankings right now!
#### Injuries
Filtering out players no longer available will improve the prediction power of the code.  Simply add names to the injury spreadsheet provided.  
#### Live-Draft Picks
This is the pain point.  I've made an effort to minimize the work needed by the user, but each draft is live, and you need to know what the composition of every team is as it's happening.  


In [1]:
# Load packages.  Must have pandas and numpy.  

import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import copy
from GameDayFunctions.fangraphs_projection_2021 import Projection
from GameDayFunctions.draft_2021 import Draft

In [2]:
autodraft_depth = 'end'
search_depth = 2
year = 2021
path_data = "projections/"
path_drafts = "Draft_Pick_Spreadsheets/"
shuffle_picks = True
ranking_method = 'FantasyPros'

First need a **Projection** object.  Projections are scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips), and include ZiPS, Steamer, and TheBat.  
In this example I create a dictionary of the three projections.  

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS',ranking_method = ranking_method)
player_rankings = {"ZiPS":player_ranking_zips}

projections/2021/ZiPS_2021_Pitchers.csv
projections/2021/ZiPS_2021_Hitters.csv


Enter your roto-league-specific stats.  The default settings are shown here.  

In [4]:
number_teams = 12
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':1}
batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] 
draft_position = 8

# Choose the projection system you prefer ()
zips = Draft(player_rankings['ZiPS'], draft_position = draft_position, number_teams = number_teams,roster_spots = roster_spots,batter_stats = batter_stats,pitcher_stats = pitcher_stats)
#steamer_6 = Draft(player_rankings['Steamer'], draft_position = draft_position, number_teams = number_teams,roster_spots = roster_spots,batter_stats = batter_stats,pitcher_stats = pitcher_stats)
#thebat_6 = Draft(player_rankings['TheBat'], draft_position = draft_position, number_teams = number_teams,roster_spots = roster_spots,batter_stats = batter_stats,pitcher_stats = pitcher_stats)

In [5]:
draft_pick_file = zips.start_new_draft()

In [6]:
draft_pick_file

'LiveDraft__FantasyPros_03192021.xlsx'

In [7]:
player_to_pick, projected_roto_stats = zips.draft_from_list_and_find_best_pick(search_depth = 2, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

IndexError: single positional indexer is out-of-bounds

###### 

In [5]:
def print_draft_positions(draft_position, number_players = None, number_teams = 12, roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':1}
):
    nrounds = sum(roster_spots.values())
    if number_players == None:
        number_players = nrounds * number_teams
        
    cnt = 0
    draft_positions = []
    while cnt < number_players:
        for iround in np.arange(nrounds):
            # Reverse draft order every other round
            draft_order = np.arange(1,number_teams+1)
            iter_team = 1
            if iround % 2 == 1:
                draft_order = draft_order[::-1]
                iter_team = 0

            for i in np.arange(number_teams):
                draft_positions.append(draft_order[i])
                cnt = cnt + 1
                if cnt == number_players:
                    break
            if cnt == number_players:
                    break
        
    return draft_positions

In [6]:
tst = print_draft_positions(8)

In [7]:
from datetime import date

def start_new_draft(path_draft = '/home/marcoviero/Repositories/GameDay2021/Draft_Pick_Spreadsheets/',path_model = '/home/marcoviero/Repositories/GameDay2021/projections/2021/PositionalRankings/', model='FantasyPros', year=2021, overwrite=False):
    file = model+'_Roto_Ranking_'+str(year)+'.xlsx'
    df = pd.read_excel(os.path.join(path_model,model,file))
    df['Picks'] = ''
    dp = print_draft_positions(8, number_players=len(df))
    df['Team'] = dp
    df=df[['RANK','Team','Picks','PLAYER','AVG','STD DEV','Elig. Pos.']]

    pdb.set_trace()
    today = date.today()
    suf = '_'+model+'_'+today.strftime("%m%d%Y")
    ending = '.xlsx'
    fileout = 'LiveDraft_'+suf+'.xlsx'
    if os.path.isfile(os.path.join(path_draft,fileout)) == False or overwrite == True :
        df.to_excel(os.path.join(path_draft,fileout),index=False)

In [ ]:
start_new_draft()

> <ipython-input-136-53bac38c199f>(12)start_new_draft()
     10 
     11     pdb.set_trace()
---> 12     today = date.today()
     13     suf = '_'+model+'_'+today.strftime("%m%d%Y")
     14     ending = '.xlsx'



ipdb>  df.head()


   RANK  Team Picks              PLAYER  AVG  STD DEV Elig. Pos.
0     1     1          Ronald Acuna Jr.  1.5      1.0   LF/CF/RF
1     2     2              Mookie Betts  2.9      1.0      CF/RF
2     3     3        Fernando Tatis Jr.  3.0      1.3         SS
3     4     4                 Juan Soto  4.0      1.5      LF/RF
4     5     5                Mike Trout  5.0      1.5         CF


ipdb>  df.head(15)


    RANK  Team Picks              PLAYER   AVG  STD DEV Elig. Pos.
0      1     1          Ronald Acuna Jr.   1.5      1.0   LF/CF/RF
1      2     2              Mookie Betts   2.9      1.0      CF/RF
2      3     3        Fernando Tatis Jr.   3.0      1.3         SS
3      4     4                 Juan Soto   4.0      1.5      LF/RF
4      5     5                Mike Trout   5.0      1.5         CF
5      6     6              Jacob deGrom   6.2      1.7         SP
6      7     7               Trea Turner   7.7      1.4         SS
7      8     8               Gerrit Cole   8.5      2.4         SP
8      9     9          Christian Yelich   9.8      1.9      LF/RF
9     10    10              Trevor Story  10.3      2.1         SS
10    11    11              Shane Bieber  10.5      2.6         SP
11    12    12              Jose Ramirez  10.8      2.2         3B
12    13    12           Freddie Freeman  12.2      1.4         1B
13    14    11              Bryce Harper  15.7      1.7      R

ipdb>  c


In [10]:
draft_pick_file

'LiveDraft__FantasyPros_03072021.xlsx'

To use while drafting in real-time, simply record the draft results in a spreadsheet (.xlsx, so if you're using Numbers, need to export to Excel) and run the cell. 

Examples below show results for after 2, 3, and 4 rounds.  Notice that at each stage the projected placement in the standings improves.  

In [87]:
player_to_pick, projected_roto_stats = zips.draft_from_list_and_find_best_pick(search_depth = 2, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 9
Stored Result for Pick 0 [16/16] Francisco Lindor SS whose placement/score is 9/120.5
Stored Result for Pick 1 [16/17] Manny Machado 3B whose placement/score is 11/105.0
Stored Result for Pick 2 [16/18] Trevor Bauer SP whose placement/score is 11/106.5
Stored Result for Pick 3 [16/19] Yu Darvish SP whose placement/score is 11/108.0
Stored Result for Pick 4 [16/23] Bo Bichette SS whose placement/score is 11/114.5
Stored Result for Pick 5 [16/24] Anthony Rendon 3B whose placement/score is 10/112.5
Stored Result for Pick 6 [16/30] Jose Abreu 1B whose placement/score is 9/117.0
Stored Result for Pick 7 [16/32] Ozzie Albies 2B whose placement/score is 10/112.5
Stored Result for Pick 8 [16/34] DJ LeMahieu 1B whose placement/score is 12/85.0
Stored Result for Pick 9 [16/36] Kyle Tucker OF whose placement/score is 11/101.5
Stored Result for Pick 10 [16/38] Eloy Jimenez OF whose placement/score is 12/94.5
Stored Result for Pick 11 [16/44] Whit Merrifield 2B whose pl

In [5]:
# Example pick in the 1st round

draft_pick_file = 'MockDraftFP_Round_1.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 2, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [6/3] Fernando Tatis SS whose placement/score is 8/120.0
Stored Result for Pick 1 [6/5] Juan Soto  OF whose placement/score is 6/132.0
Stored Result for Pick 2 [6/6] Jacob deGrom  SP whose placement/score is 6/129.0
Stored Result for Pick 3 [6/7] Trea Turner  SS whose placement/score is 7/132.5
Stored Result for Pick 4 [6/8] Gerrit Cole  SP whose placement/score is 6/122.0
Stored Result for Pick 5 [6/11] Jose Ramirez  3B whose placement/score is 2/152.5
Stored Result for Pick 6 [6/13] Freddie Freeman  1B whose placement/score is 5/139.5
Stored Result for Pick 7 [6/15] Bryce Harper  OF whose placement/score is 5/136.5
Stored Result for Pick 8 [6/17] Manny Machado  3B whose placement/score is 5/140.0
Stored Result for Pick 9 [6/28] Jose Abreu  1B whose placement/score is 12/93.0
Stored Result for Pick 10 [6/30] Ozzie Albies  2B whose placement/score is 5/132.0
Stored Result for Pick 11 [6/33] DJ LeMahieu  2B whose placement/score is 1

In [6]:
# Example pick in the 1st round

draft_pick_file = 'MockDraftFP_Round_1.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 2, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [6/3] Fernando Tatis SS whose placement/score is 2/152.5
Stored Result for Pick 1 [6/5] Juan Soto  OF whose placement/score is 7/128.5
Stored Result for Pick 2 [6/6] Jacob deGrom  SP whose placement/score is 5/139.0
Stored Result for Pick 3 [6/7] Trea Turner  SS whose placement/score is 7/126.0
Stored Result for Pick 4 [6/8] Gerrit Cole  SP whose placement/score is 6/133.5
Stored Result for Pick 5 [6/11] Jose Ramirez  3B whose placement/score is 8/127.5
Stored Result for Pick 6 [6/13] Freddie Freeman  1B whose placement/score is 8/129.5
Stored Result for Pick 7 [6/15] Bryce Harper  OF whose placement/score is 10/123.5
Stored Result for Pick 8 [6/17] Manny Machado  3B whose placement/score is 8/127.0
Stored Result for Pick 9 [6/28] Jose Abreu  1B whose placement/score is 12/77.0
Stored Result for Pick 10 [6/30] Ozzie Albies  2B whose placement/score is 7/130.0
Stored Result for Pick 11 [6/33] DJ LeMahieu  2B whose placement/score is 

In [7]:
# Example pick in the 2nd round
draft_pick_file = 'MockDraftFP_Round_2.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 2, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [19/3] Fernando Tatis SS whose placement/score is 3/143.5
Stored Result for Pick 1 [19/11] Jose Ramirez  3B whose placement/score is 3/140.5
Stored Result for Pick 2 [19/12] Shane Bieber  SP whose placement/score is 5/133.0
Stored Result for Pick 3 [19/15] Bryce Harper  OF whose placement/score is 3/142.0
Stored Result for Pick 4 [19/17] Manny Machado  3B whose placement/score is 11/116.0
Stored Result for Pick 5 [19/18] Trevor Bauer  SP whose placement/score is 11/111.5
Stored Result for Pick 6 [19/20] Xander Bogaerts  SS whose placement/score is 11/108.5
Stored Result for Pick 7 [19/28] Jose Abreu  1B whose placement/score is 11/100.5
Stored Result for Pick 8 [19/30] Ozzie Albies  2B whose placement/score is 10/114.0
Stored Result for Pick 9 [19/32] Kyle Tucker  OF whose placement/score is 9/116.5
Stored Result for Pick 10 [19/33] DJ LeMahieu  1B whose placement/score is 11/99.5
Stored Result for Pick 11 [19/41] Whit Merrifield  2

In [8]:
# Example pick in the 2nd round
draft_pick_file = 'MockDraftFP_Round_2.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 3, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [19/6] Anthony Rendon 3B whose placement/score is 12/90.0
Stored Result for Pick 1 [19/15] Whit Merrifield 2B whose placement/score is 12/87.0
Stored Result for Pick 2 [19/16] Charlie Blackmon OF whose placement/score is 12/92.0
Stored Result for Pick 3 [19/18] Fernando Tatis SS whose placement/score is 12/83.5
Stored Result for Pick 4 [19/19] Jose Altuve 2B whose placement/score is 12/87.5
Stored Result for Pick 5 [19/21] Xander Bogaerts SS whose placement/score is 12/86.5
Stored Result for Pick 6 [19/22] Jack Flaherty SP whose placement/score is 12/83.5
Stored Result for Pick 7 [19/24] Ketel Marte 2B whose placement/score is 12/88.5
Stored Result for Pick 8 [19/25] Rafael Devers 3B whose placement/score is 9/111.5
Stored Result for Pick 9 [19/27] Roberto Osuna RP whose placement/score is 9/116.5
Stored Result for Pick 10 [19/28] Bryan Reynolds OF whose placement/score is 12/99.0
Stored Result for Pick 11 [19/29] Yordan Alvarez OF 

In [11]:
# Example pick in the 3rd round

draft_pick_file = 'MockDraftFP_Round_3.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 3, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [30/15] Whit Merrifield 2B whose placement/score is 6/143.0
Stored Result for Pick 1 [30/16] Charlie Blackmon OF whose placement/score is 4/145.0
Stored Result for Pick 2 [30/19] Jose Altuve 2B whose placement/score is 6/142.0
Stored Result for Pick 3 [30/21] Xander Bogaerts SS whose placement/score is 6/139.5
Stored Result for Pick 4 [30/24] Ketel Marte 2B whose placement/score is 2/159.0
Stored Result for Pick 5 [30/27] Roberto Osuna RP whose placement/score is 7/129.5
Stored Result for Pick 6 [30/28] Bryan Reynolds OF whose placement/score is 8/110.0
Stored Result for Pick 7 [30/31] Ozzie Albies 2B whose placement/score is 2/155.5
Stored Result for Pick 8 [30/32] Bo Bichette SS whose placement/score is 8/120.5
Stored Result for Pick 9 [30/33] Josh Hader RP whose placement/score is 7/140.5
Stored Result for Pick 10 [30/35] Austin Meadows OF whose placement/score is 5/146.0
Stored Result for Pick 11 [30/36] Kirby Yates RP whose pla

In [12]:
# Example pick in the 3rd round

draft_pick_file = 'MockDraftFP_Round_3.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 3, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [30/15] Whit Merrifield 2B whose placement/score is 6/141.5
Stored Result for Pick 1 [30/16] Charlie Blackmon OF whose placement/score is 6/135.0
Stored Result for Pick 2 [30/19] Jose Altuve 2B whose placement/score is 6/137.0
Stored Result for Pick 3 [30/21] Xander Bogaerts SS whose placement/score is 7/132.0
Stored Result for Pick 4 [30/24] Ketel Marte 2B whose placement/score is 4/146.0
Stored Result for Pick 5 [30/27] Roberto Osuna RP whose placement/score is 9/126.0
Stored Result for Pick 6 [30/28] Bryan Reynolds OF whose placement/score is 10/105.0
Stored Result for Pick 7 [30/31] Ozzie Albies 2B whose placement/score is 5/136.5
Stored Result for Pick 8 [30/32] Bo Bichette SS whose placement/score is 10/105.0
Stored Result for Pick 9 [30/33] Josh Hader RP whose placement/score is 7/131.0
Stored Result for Pick 10 [30/35] Austin Meadows OF whose placement/score is 6/134.0
Stored Result for Pick 11 [30/36] Kirby Yates RP whose p

In [13]:
# Example pick in the 4nd round

draft_pick_file = 'MockDraftFP_Round_4.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [43/15] Whit Merrifield 2B whose placement/score is 1/156.0
Stored Result for Pick 1 [43/27] Roberto Osuna RP whose placement/score is 1/153.0
Stored Result for Pick 2 [43/28] Bryan Reynolds OF whose placement/score is 4/145.0
Stored Result for Pick 3 [43/32] Bo Bichette SS whose placement/score is 2/158.5
Stored Result for Pick 4 [43/33] Josh Hader RP whose placement/score is 2/152.5
Stored Result for Pick 5 [43/35] Austin Meadows OF whose placement/score is 2/155.5
Stored Result for Pick 6 [43/36] Kirby Yates RP whose placement/score is 2/149.5
Stored Result for Pick 7 [43/38] Kris Bryant 3B whose placement/score is 4/139.5
Stored Result for Pick 8 [43/40] Keston Hiura 2B whose placement/score is 2/154.0
Stored Result for Pick 9 [43/41] Tommy Pham OF whose placement/score is 5/144.0
Stored Result for Pick 10 [43/42] Aroldis Chapman RP whose placement/score is 4/145.0
Stored Result for Pick 11 [43/45] DJ LeMahieu 1B whose placement

In [14]:
# Example pick in the 4nd round

draft_pick_file = 'MockDraftFP_Round_4.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [43/15] Whit Merrifield 2B whose placement/score is 1/164.0
Stored Result for Pick 1 [43/27] Roberto Osuna RP whose placement/score is 2/153.5
Stored Result for Pick 2 [43/28] Bryan Reynolds OF whose placement/score is 2/156.0
Stored Result for Pick 3 [43/32] Bo Bichette SS whose placement/score is 5/144.5
Stored Result for Pick 4 [43/33] Josh Hader RP whose placement/score is 6/137.5
Stored Result for Pick 5 [43/35] Austin Meadows OF whose placement/score is 1/152.0
Stored Result for Pick 6 [43/36] Kirby Yates RP whose placement/score is 6/137.5
Stored Result for Pick 7 [43/38] Kris Bryant 3B whose placement/score is 9/129.5
Stored Result for Pick 8 [43/40] Keston Hiura 2B whose placement/score is 1/157.5
Stored Result for Pick 9 [43/41] Tommy Pham OF whose placement/score is 6/137.0
Stored Result for Pick 10 [43/42] Aroldis Chapman RP whose placement/score is 5/141.5
Stored Result for Pick 11 [43/45] DJ LeMahieu 1B whose placement

In [15]:
# Example pick in the 5th round

draft_pick_file = 'MockDraftFP_Round_5.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [54/27] Roberto Osuna RP whose placement/score is 4/150.0
Stored Result for Pick 1 [54/28] Bryan Reynolds OF whose placement/score is 5/145.5
Stored Result for Pick 2 [54/32] Bo Bichette SS whose placement/score is 6/144.0
Stored Result for Pick 3 [54/33] Josh Hader RP whose placement/score is 2/162.5
Stored Result for Pick 4 [54/35] Austin Meadows OF whose placement/score is 5/146.5
Stored Result for Pick 5 [54/36] Kirby Yates RP whose placement/score is 2/159.5
Stored Result for Pick 6 [54/40] Keston Hiura 2B whose placement/score is 2/160.0
Stored Result for Pick 7 [54/41] Tommy Pham OF whose placement/score is 5/148.5
Stored Result for Pick 8 [54/42] Aroldis Chapman RP whose placement/score is 3/151.5
Stored Result for Pick 9 [54/45] DJ LeMahieu 1B whose placement/score is 6/138.0
Stored Result for Pick 10 [54/47] Yoan Moncada 3B whose placement/score is 7/141.0
Stored Result for Pick 11 [54/48] Tyler Glasnow SP whose placement/

In [16]:
# Example pick in the 5th round

draft_pick_file = 'MockDraftFP_Round_5.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [54/27] Roberto Osuna RP whose placement/score is 6/142.0
Stored Result for Pick 1 [54/28] Bryan Reynolds OF whose placement/score is 5/140.5
Stored Result for Pick 2 [54/32] Bo Bichette SS whose placement/score is 6/134.0
Stored Result for Pick 3 [54/33] Josh Hader RP whose placement/score is 6/140.0
Stored Result for Pick 4 [54/35] Austin Meadows OF whose placement/score is 5/146.0
Stored Result for Pick 5 [54/36] Kirby Yates RP whose placement/score is 6/140.0
Stored Result for Pick 6 [54/40] Keston Hiura 2B whose placement/score is 4/147.0
Stored Result for Pick 7 [54/41] Tommy Pham OF whose placement/score is 5/146.0
Stored Result for Pick 8 [54/42] Aroldis Chapman RP whose placement/score is 5/135.5
Stored Result for Pick 9 [54/45] DJ LeMahieu 1B whose placement/score is 7/119.5
Stored Result for Pick 10 [54/47] Yoan Moncada 3B whose placement/score is 8/124.0
Stored Result for Pick 11 [54/48] Tyler Glasnow SP whose placement/

In [17]:
# Example pick in the 5th round

draft_pick_file = 'MockDraftFP_Round_5.xlsx'
player_to_pick, projected_roto_stats = thebat_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [54/27] Roberto Osuna RP whose placement/score is 5/145.5
Stored Result for Pick 1 [54/28] Bryan Reynolds OF whose placement/score is 4/145.5
Stored Result for Pick 2 [54/32] Bo Bichette SS whose placement/score is 5/139.5
Stored Result for Pick 3 [54/33] Josh Hader RP whose placement/score is 4/152.5
Stored Result for Pick 4 [54/35] Austin Meadows OF whose placement/score is 5/151.0
Stored Result for Pick 5 [54/36] Kirby Yates RP whose placement/score is 4/151.5
Stored Result for Pick 6 [54/40] Keston Hiura 2B whose placement/score is 4/153.0
Stored Result for Pick 7 [54/41] Tommy Pham OF whose placement/score is 4/154.0
Stored Result for Pick 8 [54/42] Aroldis Chapman RP whose placement/score is 3/154.0
Stored Result for Pick 9 [54/45] DJ LeMahieu 1B whose placement/score is 6/136.0
Stored Result for Pick 10 [54/47] Yoan Moncada 3B whose placement/score is 5/141.0
Stored Result for Pick 11 [54/48] Tyler Glasnow SP whose placement/

In [18]:
# Example pick in the 6th round

draft_pick_file = 'MockDraftFP_Round_6.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [67/27] Roberto Osuna RP whose placement/score is 7/128.5
Stored Result for Pick 1 [67/28] Bryan Reynolds OF whose placement/score is 6/143.5
Stored Result for Pick 2 [67/32] Bo Bichette SS whose placement/score is 7/127.5
Stored Result for Pick 3 [67/40] Keston Hiura 2B whose placement/score is 3/158.0
Stored Result for Pick 4 [67/41] Tommy Pham OF whose placement/score is 4/153.0
Stored Result for Pick 5 [67/42] Aroldis Chapman RP whose placement/score is 3/147.5
Stored Result for Pick 6 [67/43] Taylor Rogers RP whose placement/score is 3/148.0
Stored Result for Pick 7 [67/44] Seth Lugo RP whose placement/score is 3/149.0
Stored Result for Pick 8 [67/45] DJ LeMahieu 1B whose placement/score is 4/153.5
Stored Result for Pick 9 [67/47] Yoan Moncada 3B whose placement/score is 2/162.5
Stored Result for Pick 10 [67/48] Tyler Glasnow SP whose placement/score is 2/162.0
Stored Result for Pick 11 [67/55] Carlos Santana 1B whose placement

In [19]:
# Example pick in the 6th round

draft_pick_file = 'MockDraftFP_Round_6.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [67/27] Roberto Osuna RP whose placement/score is 9/110.0
Stored Result for Pick 1 [67/28] Bryan Reynolds OF whose placement/score is 5/140.0
Stored Result for Pick 2 [67/32] Bo Bichette SS whose placement/score is 11/104.5
Stored Result for Pick 3 [67/40] Keston Hiura 2B whose placement/score is 5/144.0
Stored Result for Pick 4 [67/41] Tommy Pham OF whose placement/score is 6/133.5
Stored Result for Pick 5 [67/42] Aroldis Chapman RP whose placement/score is 5/140.5
Stored Result for Pick 6 [67/43] Taylor Rogers RP whose placement/score is 5/139.5
Stored Result for Pick 7 [67/44] Seth Lugo RP whose placement/score is 5/137.5
Stored Result for Pick 8 [67/45] DJ LeMahieu 1B whose placement/score is 6/132.0
Stored Result for Pick 9 [67/47] Yoan Moncada 3B whose placement/score is 5/141.5
Stored Result for Pick 10 [67/48] Tyler Glasnow SP whose placement/score is 5/147.5
Stored Result for Pick 11 [67/55] Carlos Santana 1B whose placemen

In [21]:
# Example pick in the 6th round

draft_pick_file = 'MockDraftFP_Round_6.xlsx'
player_to_pick, projected_roto_stats = thebat_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [67/27] Roberto Osuna RP whose placement/score is 7/121.0
Stored Result for Pick 1 [67/28] Bryan Reynolds OF whose placement/score is 4/146.0
Stored Result for Pick 2 [67/32] Bo Bichette SS whose placement/score is 10/115.5
Stored Result for Pick 3 [67/40] Keston Hiura 2B whose placement/score is 1/160.0
Stored Result for Pick 4 [67/41] Tommy Pham OF whose placement/score is 3/151.0
Stored Result for Pick 5 [67/42] Aroldis Chapman RP whose placement/score is 5/146.0
Stored Result for Pick 6 [67/43] Taylor Rogers RP whose placement/score is 5/144.5
Stored Result for Pick 7 [67/44] Seth Lugo RP whose placement/score is 5/144.5
Stored Result for Pick 8 [67/45] DJ LeMahieu 1B whose placement/score is 5/147.5
Stored Result for Pick 9 [67/47] Yoan Moncada 3B whose placement/score is 1/160.0
Stored Result for Pick 10 [67/48] Tyler Glasnow SP whose placement/score is 3/156.0
Stored Result for Pick 11 [67/55] Carlos Santana 1B whose placemen

In [22]:
# Example pick in the 7th round

draft_pick_file = 'MockDraftFP_Round_7.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [78/27] Roberto Osuna RP whose placement/score is 7/131.5
Stored Result for Pick 1 [78/28] Bryan Reynolds OF whose placement/score is 4/148.0
Stored Result for Pick 2 [78/41] Tommy Pham OF whose placement/score is 4/151.5
Stored Result for Pick 3 [78/43] Taylor Rogers RP whose placement/score is 9/128.0
Stored Result for Pick 4 [78/44] Seth Lugo RP whose placement/score is 9/128.5
Stored Result for Pick 5 [78/50] Keone Kela RP whose placement/score is 9/124.0
Stored Result for Pick 6 [78/55] Carlos Santana 1B whose placement/score is 6/135.5
Stored Result for Pick 7 [78/59] Nick Castellanos OF whose placement/score is 9/127.5
Stored Result for Pick 8 [78/60] Jorge Polanco SS whose placement/score is 8/127.0
Stored Result for Pick 9 [78/65] Josh Bell 1B whose placement/score is 6/136.5
Stored Result for Pick 10 [78/66] Drew Pomeranz SP whose placement/score is 6/132.0
Stored Result for Pick 11 [78/67] Michael Conforto OF whose placem

In [23]:
# Example pick in the 7th round

draft_pick_file = 'MockDraftFP_Round_7.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [78/27] Roberto Osuna RP whose placement/score is 10/112.5
Stored Result for Pick 1 [78/28] Bryan Reynolds OF whose placement/score is 6/136.5
Stored Result for Pick 2 [78/41] Tommy Pham OF whose placement/score is 6/136.0
Stored Result for Pick 3 [78/43] Taylor Rogers RP whose placement/score is 10/111.0
Stored Result for Pick 4 [78/44] Seth Lugo RP whose placement/score is 10/110.5
Stored Result for Pick 5 [78/50] Keone Kela RP whose placement/score is 10/111.5
Stored Result for Pick 6 [78/55] Carlos Santana 1B whose placement/score is 7/129.5
Stored Result for Pick 7 [78/59] Nick Castellanos OF whose placement/score is 10/115.0
Stored Result for Pick 8 [78/60] Jorge Polanco SS whose placement/score is 9/117.5
Stored Result for Pick 9 [78/65] Josh Bell 1B whose placement/score is 8/127.5
Stored Result for Pick 10 [78/66] Drew Pomeranz SP whose placement/score is 10/113.5
Stored Result for Pick 11 [78/67] Michael Conforto OF whose 

In [24]:
# Example pick in the 7th round

draft_pick_file = 'MockDraftFP_Round_7.xlsx'
player_to_pick, projected_roto_stats = thebat_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [78/27] Roberto Osuna RP whose placement/score is 9/118.0
Stored Result for Pick 1 [78/28] Bryan Reynolds OF whose placement/score is 3/147.0
Stored Result for Pick 2 [78/41] Tommy Pham OF whose placement/score is 3/146.0
Stored Result for Pick 3 [78/43] Taylor Rogers RP whose placement/score is 10/115.0
Stored Result for Pick 4 [78/44] Seth Lugo RP whose placement/score is 10/112.5
Stored Result for Pick 5 [78/50] Keone Kela RP whose placement/score is 10/113.5
Stored Result for Pick 6 [78/55] Carlos Santana 1B whose placement/score is 8/130.5
Stored Result for Pick 7 [78/59] Nick Castellanos OF whose placement/score is 10/115.5
Stored Result for Pick 8 [78/60] Jorge Polanco SS whose placement/score is 9/118.5
Stored Result for Pick 9 [78/65] Josh Bell 1B whose placement/score is 7/138.5
Stored Result for Pick 10 [78/66] Drew Pomeranz SP whose placement/score is 8/121.5
Stored Result for Pick 11 [78/67] Michael Conforto OF whose pl

In [25]:
# Example pick in the 8th round

draft_pick_file = 'MockDraftFP_Round_8.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [91/28] Bryan Reynolds OF whose placement/score is 2/148.0
Stored Result for Pick 1 [91/43] Taylor Rogers RP whose placement/score is 3/149.5
Stored Result for Pick 2 [91/44] Seth Lugo RP whose placement/score is 2/150.0
Stored Result for Pick 3 [91/50] Keone Kela RP whose placement/score is 5/144.0
Stored Result for Pick 4 [91/53] Brandon Workman RP whose placement/score is 6/142.0
Stored Result for Pick 5 [91/55] Carlos Santana 1B whose placement/score is 6/143.5
Stored Result for Pick 6 [91/60] Jorge Polanco SS whose placement/score is 2/151.0
Stored Result for Pick 7 [91/66] Drew Pomeranz SP whose placement/score is 6/142.0
Stored Result for Pick 8 [91/67] Michael Conforto OF whose placement/score is 1/164.0
Stored Result for Pick 9 [91/78] Jeff McNeil 2B whose placement/score is 2/157.0
Stored Result for Pick 10 [91/85] Hunter Dozier 1B whose placement/score is 7/138.0
Stored Result for Pick 11 [91/86] Andrew Benintendi OF whos

In [26]:
# Example pick in the 8th round

draft_pick_file = 'MockDraftFP_Round_8.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [91/28] Bryan Reynolds OF whose placement/score is 2/157.0
Stored Result for Pick 1 [91/43] Taylor Rogers RP whose placement/score is 5/139.5
Stored Result for Pick 2 [91/44] Seth Lugo RP whose placement/score is 5/140.0
Stored Result for Pick 3 [91/50] Keone Kela RP whose placement/score is 5/139.5
Stored Result for Pick 4 [91/53] Brandon Workman RP whose placement/score is 6/135.5
Stored Result for Pick 5 [91/55] Carlos Santana 1B whose placement/score is 4/145.0
Stored Result for Pick 6 [91/60] Jorge Polanco SS whose placement/score is 5/141.0
Stored Result for Pick 7 [91/66] Drew Pomeranz SP whose placement/score is 5/138.5
Stored Result for Pick 8 [91/67] Michael Conforto OF whose placement/score is 2/158.0
Stored Result for Pick 9 [91/78] Jeff McNeil 2B whose placement/score is 2/153.0
Stored Result for Pick 10 [91/85] Hunter Dozier 1B whose placement/score is 6/134.5
Stored Result for Pick 11 [91/86] Andrew Benintendi OF whos

In [27]:
# Example pick in the 9th round

draft_pick_file = 'MockDraftFP_Round_9.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [102/43] Taylor Rogers RP whose placement/score is 1/165.0
Stored Result for Pick 1 [102/44] Seth Lugo RP whose placement/score is 1/165.5
Stored Result for Pick 2 [102/50] Keone Kela RP whose placement/score is 1/163.5
Stored Result for Pick 3 [102/53] Brandon Workman RP whose placement/score is 1/163.5
Stored Result for Pick 4 [102/55] Carlos Santana 1B whose placement/score is 1/171.5
Stored Result for Pick 5 [102/60] Jorge Polanco SS whose placement/score is 1/172.5
Stored Result for Pick 6 [102/66] Drew Pomeranz SP whose placement/score is 1/167.0
Stored Result for Pick 7 [102/85] Hunter Dozier 1B whose placement/score is 1/178.0
Stored Result for Pick 8 [102/108] Nate Pearson SP whose placement/score is 1/170.0
Stored Result for Pick 9 [102/120] Trey Mancini 1B whose placement/score is 1/172.0
Stored Result for Pick 10 [102/132] Diego Castillo SP whose placement/score is 2/157.0
Stored Result for Pick 11 [102/144] Jose Leclerc

In [28]:
# Example pick in the 9th round

draft_pick_file = 'MockDraftFP_Round_9.xlsx'
player_to_pick, projected_roto_stats = steamer_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [102/43] Taylor Rogers RP whose placement/score is 2/157.5
Stored Result for Pick 1 [102/44] Seth Lugo RP whose placement/score is 2/156.5
Stored Result for Pick 2 [102/50] Keone Kela RP whose placement/score is 2/156.0
Stored Result for Pick 3 [102/53] Brandon Workman RP whose placement/score is 3/153.0
Stored Result for Pick 4 [102/55] Carlos Santana 1B whose placement/score is 1/162.5
Stored Result for Pick 5 [102/60] Jorge Polanco SS whose placement/score is 1/160.5
Stored Result for Pick 6 [102/66] Drew Pomeranz SP whose placement/score is 1/159.0
Stored Result for Pick 7 [102/85] Hunter Dozier 1B whose placement/score is 1/162.5
Stored Result for Pick 8 [102/108] Nate Pearson SP whose placement/score is 1/161.5
Stored Result for Pick 9 [102/120] Trey Mancini 1B whose placement/score is 1/167.5
Stored Result for Pick 10 [102/132] Diego Castillo SP whose placement/score is 3/156.5
Stored Result for Pick 11 [102/144] Jose Leclerc

In [29]:
# Example pick in the 9th round

draft_pick_file = 'MockDraftFP_Round_9.xlsx'
player_to_pick, projected_roto_stats = thebat_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 6
Stored Result for Pick 0 [102/43] Taylor Rogers RP whose placement/score is 2/155.0
Stored Result for Pick 1 [102/44] Seth Lugo RP whose placement/score is 2/151.5
Stored Result for Pick 2 [102/50] Keone Kela RP whose placement/score is 3/150.0
Stored Result for Pick 3 [102/53] Brandon Workman RP whose placement/score is 2/150.5
Stored Result for Pick 4 [102/55] Carlos Santana 1B whose placement/score is 2/154.0
Stored Result for Pick 5 [102/60] Jorge Polanco SS whose placement/score is 2/157.0
Stored Result for Pick 6 [102/66] Drew Pomeranz SP whose placement/score is 3/151.0
Stored Result for Pick 7 [102/85] Hunter Dozier 1B whose placement/score is 1/163.5
Stored Result for Pick 8 [102/108] Nate Pearson SP whose placement/score is 1/161.0
Stored Result for Pick 9 [102/120] Trey Mancini 1B whose placement/score is 1/165.0
Stored Result for Pick 10 [102/132] Diego Castillo SP whose placement/score is 4/152.5
Stored Result for Pick 11 [102/144] Jose Leclerc

In [ ]:
# Example pick in the 10th round

draft_pick_file = 'MockDraftFP_Round_10.xlsx'
player_to_pick, projected_roto_stats = zips_16.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [10]:
# Example pick in the 11th round

draft_pick_file = 'MockDraftFP_Round_11.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 7
Stored Result for Pick 0 [127/50] Keone Kela RP whose placement/score is [6, 148.5]
Stored Result for Pick 1 [127/53] Brandon Workman P whose placement/score is [4, 139.0]
Stored Result for Pick 2 [127/55] Carlos Santana 1B whose placement/score is [2, 132.0]
Stored Result for Pick 3 [127/66] Drew Pomeranz SP whose placement/score is [4, 140.5]
Stored Result for Pick 4 [127/85] Hunter Dozier 3B whose placement/score is [3, 87.0]
Stored Result for Pick 5 [127/88] Michael Brantley OF whose placement/score is [1, 136.5]
Stored Result for Pick 6 [127/107] Tommy Edman 2B whose placement/score is [3, 87.0]
Stored Result for Pick 7 [127/110] Shogo Akiyama UTIL whose placement/score is [9, 138.0]
Stored Result for Pick 8 [127/157] Elvis Andrus SS whose placement/score is [2, 132.0]
Not Storing Result for Pick 9 [127/492] Wilson Ramos C
Best Pick is Michael Brantley putting you in 1 place


In [9]:
# Example pick in the 12th round

draft_pick_file = 'MockDraftFP_Round_12.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 7
Stored Result for Pick 0 [138/50] Keone Kela RP whose placement/score is [5, 97.5]
Stored Result for Pick 1 [138/53] Brandon Workman P whose placement/score is [4, 134.5]
Stored Result for Pick 2 [138/66] Drew Pomeranz SP whose placement/score is [2, 141.5]
Stored Result for Pick 3 [138/85] Hunter Dozier 1B whose placement/score is [1, 133.5]
Stored Result for Pick 4 [138/107] Tommy Edman 2B whose placement/score is [1, 136.5]
Stored Result for Pick 5 [138/110] Shogo Akiyama OF whose placement/score is [8, 150.5]
Stored Result for Pick 6 [138/141] David Peralta UTIL whose placement/score is [2, 138.5]
Stored Result for Pick 7 [138/157] Elvis Andrus SS whose placement/score is [1, 135.5]
Stored Result for Pick 8 [138/167] Justin Turner 3B whose placement/score is [1, 134.5]
Not Storing Result for Pick 9 [138/492] Wilson Ramos C
Best Pick is Tommy Edman putting you in 1 place


In [8]:
# Example pick in the 13th round

draft_pick_file = 'MockDraftFP_Round_13.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 7
Stored Result for Pick 0 [151/50] Keone Kela RP whose placement/score is [7, 132.5]
Stored Result for Pick 1 [151/54] Nick Anderson P whose placement/score is [2, 162.0]
Stored Result for Pick 2 [151/85] Hunter Dozier 1B whose placement/score is [2, 156.5]
Stored Result for Pick 3 [151/107] Tommy Edman 2B whose placement/score is [2, 157.5]
Stored Result for Pick 4 [151/108] Nate Pearson SP whose placement/score is [3, 96.0]
Stored Result for Pick 5 [151/110] Shogo Akiyama OF whose placement/score is [7, 130.0]
Stored Result for Pick 6 [151/141] David Peralta UTIL whose placement/score is [2, 157.0]
Stored Result for Pick 7 [151/158] Kevin Newman SS whose placement/score is [2, 156.0]
Stored Result for Pick 8 [151/167] Justin Turner 3B whose placement/score is [2, 157.0]
Not Storing Result for Pick 9 [151/492] Wilson Ramos C
Best Pick is Nick Anderson putting you in 2 place


In [ ]:
# Example pick in the 14th round

draft_pick_file = 'MockDraftFP_Round_14.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

In [19]:
# Example pick in the 15th round

draft_pick_file = 'MockDraftFP_Round_15.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 7
Stored Result for Pick 0 [175/62] Sean Doolittle BN whose placement/score is [6, 150.0]
Stored Result for Pick 1 [175/108] Nate Pearson SP whose placement/score is [6, 152.5]
Not Storing Result for Pick 2 [175/497] Francisco Mejia C
Best Pick is Nate Pearson putting you in 6 place


In [5]:
# Example pick in the 16th round

draft_pick_file = 'MockDraftFP_Round_16.xlsx'
player_to_pick, projected_roto_stats = zips_6.draft_from_list_and_find_best_pick(search_depth = 4, autodraft_depth = autodraft_depth, draft_pick_file = draft_pick_file)

Finding Best Pick For Team 7
Stored Result for Pick 0 [175/62] Sean Doolittle RP whose placement/score is [7, 129.5]
Stored Result for Pick 1 [175/79] Tyler Duffey P whose placement/score is [7, 128.0]
Stored Result for Pick 2 [175/85] Hunter Dozier 1B whose placement/score is [3, 84.5]
Stored Result for Pick 3 [175/108] Nate Pearson SP whose placement/score is [6, 154.5]
Stored Result for Pick 4 [175/110] Shogo Akiyama OF whose placement/score is [7, 128.0]
Stored Result for Pick 5 [175/141] David Peralta UTIL whose placement/score is [3, 83.5]
Stored Result for Pick 6 [175/158] Kevin Newman 2B whose placement/score is [3, 85.5]
Stored Result for Pick 7 [175/178] Jean Segura SS whose placement/score is [1, 126.0]
Stored Result for Pick 8 [175/205] Luis Arraez 3B whose placement/score is [4, 129.0]
Not Storing Result for Pick 9 [175/497] Francisco Mejia C
Best Pick is Jean Segura putting you in 1 place


In [33]:
# Projected_roto_stats contains the estimated results from all teams given the recommented pick

projected_stats = projected_roto_stats[0]
projected_standings = projected_roto_stats[3]
projected_place = projected_roto_stats[4]
projected_scores = projected_roto_stats[5]

In [34]:
# The projected_stats give the raw numbers

projected_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,642.0,694.0,222.0,21.0,193.0,623.0,56.0,488.0,0.267851,0.814452,52.0,40.0,2.0,2.0,54.0,258.0,839.0,3.817583,1.219351,4.0
1,712.0,688.0,243.0,23.0,212.0,656.0,79.0,547.0,0.276748,0.860217,42.0,35.0,2.0,2.0,24.0,275.0,808.0,3.959868,1.261534,2.0
2,864.0,816.0,285.0,28.0,304.0,869.0,102.0,574.0,0.265375,0.836066,35.0,29.0,2.0,2.0,31.0,195.0,647.0,3.896596,1.205041,3.0
3,724.0,768.0,263.0,24.0,207.0,697.0,71.0,515.0,0.263246,0.796572,72.0,44.0,5.0,5.0,53.0,291.0,1182.0,3.634691,1.152340,4.0
4,600.0,648.0,203.0,22.0,168.0,549.0,101.0,405.0,0.271512,0.812575,38.0,33.0,2.0,2.0,28.0,249.0,708.0,3.940271,1.282323,2.0
5,853.0,944.0,312.0,36.0,240.0,816.0,87.0,598.0,0.275758,0.822687,37.0,28.0,2.0,2.0,56.0,204.0,720.0,3.761795,1.198750,5.0
6,654.0,697.0,244.0,22.0,198.0,665.0,42.0,469.0,0.268858,0.818252,53.0,29.0,3.0,3.0,0.0,205.0,769.0,3.383137,1.138375,0.0
7,875.0,991.0,299.0,26.0,264.0,857.0,88.0,517.0,0.281342,0.829115,23.0,16.0,0.0,0.0,83.0,173.0,542.0,3.655680,1.207149,6.0
8,749.0,818.0,262.0,25.0,231.0,748.0,65.0,524.0,0.274292,0.827410,64.0,53.0,4.0,4.0,50.0,320.0,1081.0,3.846651,1.209197,4.0
9,768.0,810.0,278.0,28.0,226.0,721.0,107.0,467.0,0.272899,0.821081,43.0,36.0,2.0,2.0,26.0,262.0,788.0,4.015090,1.253797,2.0


In [35]:
# And projected_scores show how they translate to roto scores

projected_scores

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BBP,SO,ERA,WHIP,BSV
0,2.0,3.0,2.0,1.0,3.0,2.0,2.0,4.0,3.0,3.0,7.5,4.0,4.5,4.5,9.0,6.0,8.0,6.0,4.0,5.0
1,5.0,2.0,4.0,4.0,6.0,4.0,6.0,10.0,11.0,12.0,5.0,6.0,4.5,4.5,2.0,4.0,7.0,2.0,2.0,10.0
2,11.0,9.0,10.0,9.5,12.0,12.0,11.0,11.0,2.0,11.0,2.0,9.5,4.5,4.5,5.0,11.0,2.0,4.0,7.0,7.5
3,6.0,6.0,8.0,5.0,5.0,6.0,4.0,7.0,1.0,1.0,12.0,3.0,12.0,12.0,8.0,3.0,12.0,10.0,10.0,5.0
4,1.0,1.0,1.0,2.5,1.0,1.0,9.5,1.0,5.0,2.0,4.0,7.0,4.5,4.5,4.0,7.0,3.0,3.0,1.0,10.0
5,10.0,11.0,12.0,12.0,10.0,10.0,7.0,12.0,10.0,6.0,3.0,11.0,4.5,4.5,10.0,10.0,4.0,7.0,8.0,3.0
6,3.0,4.0,5.0,2.5,4.0,5.0,1.0,3.0,4.0,4.0,9.0,9.5,8.5,8.5,1.0,9.0,5.0,12.0,11.0,12.0
7,12.0,12.0,11.0,7.0,11.0,11.0,8.0,8.0,12.0,10.0,1.0,12.0,1.0,1.0,12.0,12.0,1.0,9.0,6.0,1.5
8,8.0,10.0,7.0,6.0,9.0,9.0,3.0,9.0,8.0,9.0,11.0,1.0,10.5,10.5,7.0,1.0,10.0,5.0,5.0,5.0
9,9.0,8.0,9.0,9.5,8.0,7.5,12.0,2.0,7.0,5.0,6.0,5.0,4.5,4.5,3.0,5.0,6.0,1.0,3.0,10.0


In [36]:
# Finally, the projected_standings shows the resulting standings

projected_standings

5     165.0
7     158.5
2     155.5
10    153.0
8     144.0
3     136.0
11    134.5
9     125.0
6     121.0
1     111.0
0      83.5
4      73.0
dtype: float64

In [37]:
# But if you just want to know how your team finishes, that's projected_place

projected_place

1